# MetaGen Quickstart

This notebook demonstrates the basic MetaGen workflow: from spec to complete release package.

## Prerequisites

Make sure MetaGen is installed:
```bash
pip install -e .
```

In [ ]:
# Imports
import json
from pathlib import Path

# MetaGen imports
from metagen.specs.loader import load_spec
from metagen.specs.schema import json_schema
from metagen.synth.engine import synthesize

## 1. View the JSON Schema

First, let's see what fields are available in a MetaGen spec:

In [ ]:
# Get the JSON schema
schema = json_schema()

# Show top-level properties
print("Top-level spec fields:")
for field in schema.get("properties", {}).keys():
    print(f"  - {field}")

## 2. Load an Example Spec

Let's load the text LLM example spec:

In [ ]:
# Path to example spec
spec_path = Path("../specs/text/text_llm_8b.yaml")

# Load and validate
spec, resolved_seed = load_spec(spec_path)

print(f"Spec: {spec.name}")
print(f"Description: {spec.description}")
print(f"Resolved seed: {resolved_seed}")
print("\nModality:")
print(f"  Inputs: {spec.modality.inputs}")
print(f"  Outputs: {spec.modality.outputs}")
print("\nConstraints:")
print(f"  Parameter budget: {spec.constraints.parameter_budget.max}")
print(f"  Context window: {spec.constraints.context_window}")

## 3. Run Synthesis

Now let's synthesize a complete release package:

In [ ]:
# Output directory
output_dir = Path("./outputs/quickstart_demo")

# Run synthesis
result = synthesize(spec=spec, output_dir=output_dir, seed=resolved_seed)

print("Synthesis complete!")
print(f"Output directory: {result['output_dir']}")

## 4. Explore the Output

Let's see what was generated:

In [ ]:
# List generated files
output_path = Path(result["output_dir"])

print("Generated files:")
for item in sorted(output_path.rglob("*")):
    if item.is_file():
        rel_path = item.relative_to(output_path)
        print(f"  {rel_path}")

## 5. View the Architecture Blueprint

Let's examine the generated architecture:

In [ ]:
import yaml

# Load architecture blueprint
arch_path = output_path / "blueprint" / "architecture.yaml"
with open(arch_path) as f:
    architecture = yaml.safe_load(f)

print("Architecture Blueprint:")
print(f"  Hidden size: {architecture.get('hidden_size')}")
print(f"  Num layers: {architecture.get('num_layers')}")
print(f"  Num heads: {architecture.get('num_heads')}")
print(f"  Vocab size: {architecture.get('vocab_size')}")

## 6. View Benchmark Scores

MetaGen generates synthetic but plausible benchmark scores:

In [ ]:
# Load summary
summary_path = output_path / "docs" / "summary.json"
with open(summary_path) as f:
    summary = json.load(f)

print("Benchmark Scores:")
for name, score in summary.get("benchmarks", {}).items():
    print(f"  {name}: {score:.3f}")

## 7. View Generated Code

Let's peek at the generated model code:

In [ ]:
# Read model.py
model_path = output_path / "code" / "model.py"
with open(model_path) as f:
    model_code = f.read()

# Show first 50 lines
lines = model_code.split("\n")[:50]
print("model.py (first 50 lines):")
print("=" * 60)
for i, line in enumerate(lines, 1):
    print(f"{i:3}: {line}")

## 8. Verify Determinism

Same spec + same seed = identical outputs:

In [ ]:
# Run synthesis again with same seed
output_dir2 = Path("./outputs/quickstart_demo_2")
result2 = synthesize(spec=spec, output_dir=output_dir2, seed=resolved_seed)

# Compare architectures
arch_path2 = Path(result2["output_dir"]) / "blueprint" / "architecture.yaml"
with open(arch_path) as f1, open(arch_path2) as f2:
    arch1 = f1.read()
    arch2 = f2.read()

print(f"Architectures identical: {arch1 == arch2}")

## Next Steps

- [02_automl_search.ipynb](02_automl_search.ipynb) - Architecture search
- [03_multi_modal.ipynb](03_multi_modal.ipynb) - Different modalities
- [Spec Language Reference](../../docs/user-guide/spec_language.md)